In [9]:
%matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
# import gmplot
# from geopy.geocoders import Nominatim
# import time
# import gmaps
# import gmaps.datasets

Aijobsindustry_dataset = pd.read_csv("AIJobsIndustry_Clean_v3.csv")
print(f"No of rows in Aijobsindustry_dataset is {Aijobsindustry_dataset.shape[0]}")

Dice_us_jobs_dataset = pd.read_csv("Dice_US_jobs_Clean_v4.csv", encoding="cp437")
print(f"No of rows in Dice_us_jobs_dataset is {Dice_us_jobs_dataset.shape[0]}")

monster_dataset = pd.read_csv("Monster_Clean.csv")
print(f"No of rows in monster_dataset is {monster_dataset.shape[0]}")

indeed_dataset = pd.read_csv("alldata_Clean_v3.csv")
print(f"No of rows in indeed_dataset is {indeed_dataset.shape[0]}")

multi_src_dataset = pd.read_csv("cleanedandfilteredUSJobs.csv")
print(f"No of rows in multiple source data set is {multi_src_dataset.shape[0]}")

##file needs to match format for concat
dice_tech_filtered = pd.read_csv("Dice_US_Tech_FilteredJobData.csv")
print(f"No of rows in dice_tech_filtered is {dice_tech_filtered.shape[0]}")

print(f"No of rows in final merged data set should be {Aijobsindustry_dataset.shape[0]+multi_src_dataset.shape[0]+Dice_us_jobs_dataset.shape[0]+monster_dataset.shape[0]+indeed_dataset.shape[0]+dice_tech_filtered.shape[0]}")

datasets_to_merge = [Aijobsindustry_dataset,Dice_us_jobs_dataset,monster_dataset,indeed_dataset,dice_tech_filtered,multi_src_dataset]

##please confirm the concat makes sense
merged_dataset = pd.concat(datasets_to_merge, sort=False)
print(f"Actual no of rows in merged_dataset is {merged_dataset.shape[0]}")

Using matplotlib backend: Qt5Agg
No of rows in Aijobsindustry_dataset is 631
No of rows in Dice_us_jobs_dataset is 1265
No of rows in monster_dataset is 162
No of rows in indeed_dataset is 6953
No of rows in multiple source data set is 207
No of rows in dice_tech_filtered is 1566
No of rows in final merged data set should be 10784
Actual no of rows in merged_dataset is 10784


In [10]:
merged_dataset.head()

,Position,Company,Description,Loc_state,Loc_city,Skills,Type,Source
0,Data Science Development Program- Junior Data ...,MassMutual,NaN,MA,Amherst,NaN,NaN,NaN
1,Entry-Level Cognitive / AI / Machine Learning ...,IBM,NaN,TX,Austin,NaN,NaN,NaN
2,PHP/MySQL Programmer,NaN,NaN,TX,Austin,NaN,NaN,NaN
3,Python Programmer Job,SAIC,NaN,CA,Monterey,NaN,NaN,NaN
4,BI Analyst,NaN,NaN,CA,San Francisco,NaN,NaN,NaN


In [11]:
merged_dataset.to_csv("final_dataframe.csv")